In [ ]:
import os
import requests
import csv
import logging 

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import emc

from rdkit import Chem
from rdkit.Chem import RDKFingerprint
from rdkit.Chem import rdMolDescriptors

from matplotlib.ticker import StrMethodFormatter

## EXTRACT DATA
CSV file to be present in local machine.<br>
FIle name is data.csv

In [ ]:
# Downloading the data from the GitHub repository
if os.path.exists("data.csv"):
    print("Data already downloaded, skipping download....")
else:
    url = 'https://github.com/sriskid/hts_active_learning/raw/main/data.csv'
    res = requests.get(url, allow_redirects=True)
    with open('data.csv','wb') as f:
        print("Downloading data....")
        f.write(res.content)

In [ ]:
data = []
with open('data.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, skipinitialspace=True)
    # print(next(reader))
    # data.append(tuple(next(reader))) # header
    
    for ID, SMILE, B, RES, is_active in reader:
        data.append((ID, SMILE, B, RES, is_active)) #append data as tuple

# remove header
data = data[1:]
maxlen = len(data)
print('Number of entries processed: ', maxlen)

In [ ]:
# ID, SMILE, B, RES, is_active
data[0]

## MOLECULAR FINGERPRINT ENCODING OF SMILES
This will help in converting SMILES (Simplified molecular-input line-entry system) data so that it can be used for applying ML techniques. This is more often used (from surveying)<br>
> Credit: https://towardsdatascience.com/basic-molecular-representation-for-machine-learning-b6be52e9ff76

RDKfingerprint()<br>
> Returns an RDKit topological fingerprint for a molecule<br>
> doc: https://www.rdkit.org/docs/source/rdkit.Chem.rdmolops.html

A fingerprint is a simplified, encoded representation of a molecule's structure that is used for various computational tasks, such as searching, comparing, and clustering molecules in large databases. The key idea behind a fingerprint is to convert the complex structure of a molecule into a fixed-size binary/bitstring vector. The conversion from a molecule to a fingerprint is irreversible.

A topological fingerprint (also known as a path-based fingerprint) is a type of molecular fingerprint where each bit represents the presence of a particular atom sequence (paths) or bond pattern (substructures) within the molecule; the connectivity of molelcules. This fingerprint encodes the connectivity of atoms (i.e., the topological structure) without considering 3D spatial information.

In [ ]:
# Drawing molecules
mol = Chem.MolFromSmiles(data[5][1])
mol

In [ ]:
# We will be using RDK fingerprint to vectorize all our SMILES structures
fingerprints = []
for i in range(len(data)):
    mol = Chem.MolFromSmiles(data[i][1])
    fingerprint_rdk = np.array(RDKFingerprint(mol))
    fingerprints.append(fingerprint_rdk)
    
fingerprints = np.array(fingerprints)
fingerprints

In [ ]:
fingerprints[0].shape

## VISUALIZATION OF DATA
Data is obtained from the following paper - https://doi.org/10.1371/journal.pcbi.1010613 <br>
In this paper, they conducted a series of HTS to obtain the Average B-score and residual values for a series of small molecules to see their potency against Burkholderia cenocepacia.<br>
Quoting the paper,<br>

"The dataset used in the ML approach consisted of 29,537 compounds with residual growth (RG) values and average B-scores. The RG measures the ratio of bacterial growth in the presence and absence of the compounds. The B-score measures relative potency that adjusts the RG for any screening artifacts resulting from well position (row and column) in the assay plate during the HTS. The B-score is inversely proportional to compound potency, where negative B-scores indicate greater growth inhibitory activity of the compounds. To binarize the compounds, the previously established average B-score threshold of -17.5 was chosen [https://doi.org/10.1371/journal.pone.0128587]. Overall, 256 small molecules were classified as growth inhibitory."

Staying true to the essence of the paper, we will be setting a threshold of -17.5 for B-score in our data as well, i.e. below this threshold, the compound is labeled active, and otherwise inactive.

In [ ]:
# Checking data for number of active compounds
# idx - ID, SMILE, B, RES, is_active
count = 0
for i in range(len(data)):
    if float(data[i][2]) <= -17.5:
        if float(data[i][4]) == 1:
            count += 1
print("Number of active compounds =",count)
# This follows what is mentioned in the paper.

In [ ]:
# Get the B-score values and labels
Bscores = []
active = []
is_active = []
sample_ids = []

for i in range(len(data)):
    Bscores.append(float(data[i][2]))

    is_active.append(int(data[i][4]))
    sample_ids.append(data[i][0])

    if float(data[i][4]) == 0:
        active.append("Inactive")
    else:
         active.append("Active")

compound_idx = np.arange(len(Bscores))
is_active = np.array(is_active)
sample_ids = np.array(sample_ids)

In [ ]:
# Visualizing data
# Plot B-scores

fig, ax = plt.subplots()
# plt.scatter(compound_idx[0:100], Bscores[0:100], c ="blue")
sns.set(style='whitegrid')
sns.scatterplot(x=compound_idx,
                    y=Bscores,
                    hue=active, marker='+')

# ax.set_ylim(-500, 100)
ax.set(xlabel='Compound Index', ylabel='B-Score')

In [ ]:
print(fingerprints.shape)
print(is_active.shape)
print(sample_ids.shape)

## Splitting Imbalanced Data 
Our data is heavily imbalanced and it is important to balance the major and minor labels. Therefore, we do random oversampling of the minor class.


In [ ]:
# Test out the oversample/split function
train_x, train_y, test_x, test_y, test_ids, train_ids = \
    emc.train_test_split_oversample(fingerprints, is_active, 
                                    oversample_type='ros', 
                                    split_ratio=0.8, 
                                    oversample_size=0.4,
                                    seed=0, 
                                    sample_ids=sample_ids)

print("Number of active compounds in:")
print("Train Set = ", sum(train_y == 1))
print("Test Set = ", sum(test_y == 1))

print("Number of inactive compounds in:")
print("Train Set = ", sum(train_y == 0))
print("Test Set = ", sum(test_y == 0))

In [ ]:
# Create small dataset for debugging

# set number of major&minor labeled samples to be included in small dataset
n_samples = 80

# identify minor label
minor_label = emc.get_minor_label(is_active)
# get each label indices and slice into small number
minor_label_idx_arr = np.where(is_active==minor_label)[0]
minor_label_idx_arr = np.random.permutation(minor_label_idx_arr)[:n_samples]
major_label_idx_arr = np.where(is_active!=minor_label)[0]
major_label_idx_arr = np.random.permutation(major_label_idx_arr)[:n_samples]

# concat both labels
small_data = np.concatenate([fingerprints[minor_label_idx_arr], fingerprints[major_label_idx_arr]])
small_label = np.concatenate([is_active[minor_label_idx_arr], is_active[major_label_idx_arr]])
small_sample_ids = np.concatenate([sample_ids[minor_label_idx_arr], sample_ids[major_label_idx_arr]])

# shuffle 
suffled_idx = np.random.permutation(len(small_data))
small_data = small_data[suffled_idx]
small_label = small_label[suffled_idx]
small_sample_ids = small_sample_ids[suffled_idx]

print('small_data shape:',small_data.shape)
print('small_label shape:',small_label.shape)
print('small_sample_ids shape:',small_sample_ids.shape)

In [ ]:
# Test/train split function on SMALL DEBUGGIN DATASET
train_x, train_y, test_x, test_y = \
    emc.train_test_split_oversample(small_data, small_label, 
                                    oversample_type='ros', 
                                    split_ratio=0.8, 
                                    oversample_size=20,
                                    seed=0)

print("Number of active compounds in:")
print("Train Set = ", sum(train_y == 1))
print("Test Set = ", sum(test_y == 1))

print("Number of inactive compounds in:")
print("Train Set = ", sum(train_y == 0))
print("Test Set = ", sum(test_y == 0))

### Model configuration

In [ ]:
# Configuration for both EMC and random sampling models
config={
    'emc_dir_path':'drug_emc_res',
    'rand_sampling_dir_path':'drug_rand_sampling_res',
    'n_sim':10,
    'initial_train_ratio':0.05,    
    # 'ros' OR 'smote' 
    'oversample_type':'smote', 
    'split_ratio':0.8,
    # ratio in range [0.0,1.0] OR number (positive int) of samples to oversample
    'oversample_size':836,
    'log_freq':50
}

test_config={
    'emc_dir_path':'emc_res',
    'rand_sampling_dir_path':'rand_sampling_res',
    'n_sim':3,
    'initial_train_ratio':0.2,
    # 'ros' OR 'smote' 
    'oversample_type':'smote', 
    'split_ratio':0.8,
    # ratio in range [0.0,1.0] OR number (positive int) of samples to oversample
    'oversample_size':0.2,
    'log_freq':4
}

### Set up results directory and logger  

In [ ]:
# Make new directory to store results

# emc sampling results directory
emc_dir_path = make_unique_file_name(config['emc_dir_path'])
os.mkdir(emc_dir_path)

# random sampling results directory
rand_sampling_dir_path = make_unique_file_name(config['rand_sampling_dir_path'])
os.mkdir(rand_sampling_dir_path)

In [ ]:
# Set up logger for emc sampling simulation
logger_emc = logging.getLogger(__name__)
logger_emc.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s %(message)s', datefmt='%Y-%m-%d %H:%M')
file_handler = logging.FileHandler(f'{emc_dir_path}/emc.log')
file_handler.setFormatter(formatter)

logger_emc.addHandler(file_handler)

In [ ]:
# Set up logger for random sampling simulation
logger_rand_sampler = logging.getLogger(__name__)
logger_rand_sampler.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s %(message)s', datefmt='%Y-%m-%d %H:%M')
file_handler = logging.FileHandler(f'{rand_sampling_dir_path}/rand_sampling.log')
file_handler.setFormatter(formatter)

logger_rand_sampler.addHandler(file_handler)

### Run Expected Model Change Simulation on Molecular Fingerprinting Data
Logistic regression will be our base learner

In [ ]:
# EMC simulation on small test case data
emc.run_l_simulations_emc(small_data, small_label, sample_ids=small_sample_ids, **test_config)

# EMC simulation on real data
# emc.run_l_simulations_emc(fingerprints, is_active, sample_ids=sample_ids, **config)

In [ ]:
# Random Sampling simulation on small test case data
emc.run_n_simulations_random_sampling(small_data, small_label, **config)

# Random Sampling simulation on real data
# emc.run_n_simulations_random_sampling(fingerprints, is_active, **config)

In [ ]:
# Load EMC results
n_sim_accuracy_ls, n_sim_precision_ls, n_sim_recall_ls = [],[],[]

for i in range(config['n_sim']):
    n_sim_accuracy_ls.append(np.load(f"{config['emc_dir_path']}/{i}_sim_emc_accuracy.npy"))
    n_sim_precision_ls.append(np.load(f"{config['emc_dir_path']}/{i}_sim_emc_precision.npy"))
    n_sim_recall_ls.append(np.load(f"{config['emc_dir_path']}/{i}_sim_emc_recall.npy"))

In [ ]:
# Load Random Sampling results
n_sim_accuracy_random_ls, n_sim_precision_random_ls, n_sim_recall_random_ls = [],[],[]

for i in range(config['n_sim']):
    n_sim_accuracy_random_ls.append(np.load(f"{config['rand_sampling_dir_path']}/{i}_sim_rand_sample_accuracy.npy"))
    n_sim_precision_random_ls.append(np.load(f"{config['rand_sampling_dir_path']}/{i}_sim_rand_sample_precision.npy"))
    n_sim_recall_random_ls.append(np.load(f"{config['rand_sampling_dir_path']}/{i}_sim_rand_sample_recall.npy"))

In [ ]:
n_updates =  len(n_sim_accuracy_ls[0])

plt.figure(figsize=(10,6))
emc.plot_metrics(n_sim_accuracy_ls, n_updates, plot_separate_sim=False, color='red', label='emc query')
emc.plot_metrics(n_sim_accuracy_random_ls, n_updates, plot_separate_sim=False, color='blue', label='random query')
plt.title('Accuracy')
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}'))
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
emc.plot_metrics(n_sim_precision_ls, n_updates, plot_separate_sim=False, color='red', label='emc query')
emc.plot_metrics(n_sim_precision_random_ls, n_updates, plot_separate_sim=False, color='blue', label='random query')
plt.title('Precision')
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}'))
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
emc.plot_metrics(n_sim_recall_ls, n_updates, plot_separate_sim=False, color='red', label='emc query')
emc.plot_metrics(n_sim_recall_random_ls, n_updates, plot_separate_sim=False, color='blue', label='random query')
plt.title('Recall')
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}'))
plt.show()

In [ ]:
# plot for github
from matplotlib.ticker import FormatStrFormatter

def metric_plotter(ax, x, emc_y, baseline_y, title):
    ax.set_title(title)
    ax.plot(x,emc_y,color='red', label='emc query')
    ax.plot(x,baseline_y,color='blue',label='random query')
    ax.legend()
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

n_updates =  len(n_sim_accuracy_ls[0])
x = np.linspace(1, n_updates, n_updates)

emc_avg_acc_arr = np.sum(n_sim_accuracy_ls,axis=0) / len(n_sim_accuracy_ls)
emc_avg_prec_arr = np.sum(n_sim_precision_ls,axis=0) / len(n_sim_precision_ls)
emc_avg_recall_arr = np.sum(n_sim_recall_ls,axis=0) / len(n_sim_recall_ls)

rand_avg_acc_arr = np.sum(n_sim_accuracy_random_ls,axis=0) / len(n_sim_accuracy_random_ls)
rand_avg_prec_arr = np.sum(n_sim_precision_random_ls,axis=0) / len(n_sim_precision_random_ls)
rand_avg_recall_arr = np.sum(n_sim_recall_random_ls,axis=0) / len(n_sim_recall_random_ls)

fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(5,13))
# plot
metric_plotter(ax1, x, emc_avg_acc_arr, rand_avg_acc_arr, title="Accuracy")
metric_plotter(ax2, x, emc_avg_prec_arr, rand_avg_prec_arr, title="Precision")
metric_plotter(ax3, x, emc_avg_recall_arr, rand_avg_recall_arr, title="Recall")